In [ ]:
import k3d
import os
from pathlib import Path
import nrrd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
x_train_path = '../dataset/raw/ASOCA2020Data/Train'
y_train_path = '../dataset/raw/ASOCA2020Data/Train_Masks'

In [ ]:
stats = []
for path in os.listdir(x_train_path):
    vol, header = nrrd.read(Path(x_train_path, path), index_order='C')
    mask, _     = nrrd.read(Path(y_train_path, path), index_order='C')
    stats.append([int(path.split('.')[0]), vol.shape, vol.sum(), vol.mean(), vol[mask==1].sum(), vol[mask==1].mean()])

In [ ]:
stats = sorted(stats, key=lambda x: x[0])

In [ ]:
np.nonzero(mask.sum(axis=-1).sum(axis=-1))

In [ ]:
fig, axes = plt.subplots(2,3, figsize=(15,10))
axes[0,0].imshow(vol[63])
axes[1,0].imshow(mask[63])
axes[0,1].imshow(vol[129])
axes[1,1].imshow(mask[129])
axes[0,2].imshow(vol[178])
axes[1,2].imshow(mask[178])
plt.tight_layout()
fig.savefig('asoca_slices.pdf', format='pdf')
plt.show()

In [ ]:
plt.imshow(vol[192])

In [ ]:
_, ax = plt.subplots(1,1, figsize=(15,7))
# ax.plot([s[2] for s in stats], label='sum all')
ax.plot([s[3] for s in stats], label='mean all')
# ax.plot([s[4] for s in stats], label='sum fg')
ax.plot([s[5] for s in stats], label='mean fg')
plt.legend()
plt.show()

In [ ]:
vol.shape

In [ ]:
vol, _ = nrrd.read(Path(x_train_path, os.listdir(x_train_path)[0]), index_order='C')
mask, _ = nrrd.read(Path(y_train_path, os.listdir(y_train_path)[0]), index_order='C')

In [ ]:
np.nonzero(mask.sum(axis=0).sum(axis=1))

In [ ]:
k3d_volume = k3d.volume(
    mask.astype(np.float32),
    alpha_coef=1000,
    shadow='dynamic',
    samples=600,
    color_map=k3d.colormaps.paraview_color_maps.Reds,
    compression_level=9
)

plot = k3d.plot(camera_auto_fit=True)
plot += k3d_volume

plot.lighting = 2
plot.display()

In [ ]:
import json
import sys
sys.path.append('..')
from data_utils.helpers import get_volume_pred
import torch
import re
import nrrd
from tqdm import tqdm

In [ ]:
with open('../dataset/3d_mnist_128/dataset.json', 'r') as f:
    meta = json.load(f)

In [ ]:
root_mnist_dir = '../dataset/3d_mnist_128'

In [ ]:
vol_paths = [Path(root_mnist_dir, 'train/vols', path) for path in os.listdir(Path(root_mnist_dir, 'train/vols'))] + \
            [Path(root_mnist_dir, 'valid/vols', path) for path in os.listdir(Path(root_mnist_dir, 'valid/vols'))]
mask_paths = [Path(root_mnist_dir, 'train/masks', path) for path in os.listdir(Path(root_mnist_dir, 'train/masks'))] + \
            [Path(root_mnist_dir, 'valid/masks', path) for path in os.listdir(Path(root_mnist_dir, 'valid/masks'))]

In [ ]:
vol_paths  = sorted(vol_paths , key=lambda x: int(re.findall('(\d+).npy', x.name)[0]))
mask_paths = sorted(mask_paths, key=lambda x: int(re.findall('(\d+).npy', x.name)[0]))

In [ ]:
output_dir = '../dataset/raw/3D_MNIST'

In [ ]:
os.makedirs(Path(output_dir,'Train'), exist_ok=True)
os.makedirs(Path(output_dir,'Train_Masks'), exist_ok=True)
os.makedirs(Path(output_dir,'Test'), exist_ok=True)

In [ ]:
def get_vol(path, meta):
    vol_meta = meta['vol_meta'][str(vol_id)]
    vol = np.load(path)
    vol_rec = get_volume_pred(
        torch.from_numpy(vol), 
        vol_meta,
        meta['patch_size'],
        meta['stride'],
        normalize=False).numpy()
    assert list(vol_rec.shape) == vol_meta['shape_orig'], f"{vol_rec.shape}, {vol_meta['shape_orig']}"
    return vol_rec

In [ ]:
def save_vol(vol_id, vol, output_dir, split):
    output_path = str(Path(output_dir, split, f'{vol_id}.nrrd'))
    header = {'space directions': np.eye(3)}
    nrrd.write(output_path, vol, header=header, index_order='C')

In [ ]:
for vol_id, path in enumerate(tqdm(vol_paths)):
    vol_rec = get_vol(path, meta)
    save_vol(vol_id, vol_rec, output_dir, 'Train')
    
for vol_id, path in enumerate(tqdm(mask_paths)):
    vol_rec = get_vol(path, meta)
    save_vol(vol_id, vol_rec, output_dir, 'Train_Masks')

In [ ]:
import shutil
shutil.make_archive('../dataset/3D_MNIST_DATA', 'zip', output_dir)

In [ ]:
np.load('../dataset/3d_mnist_128/train/masks//0.npy').shape

In [1]:
import shutil
shutil.make_archive('../dataset/processed/ASOCA2020Data/', 'zip', '../dataset/raw/')

'/home/ebekkers/damyan/ASOCA/dataset/processed/ASOCA2020Data.zip'